# 한국전력공사 
## dsz data
- dsz data는 한국전력공사가 제공하는 충전소에 한함. (~2019.12.27)

In [ ]:
#광역시도코드 / 광역시도명 /시군구코드 / 시군구명 / 설치장소명 / 주소 / 급속충전기대수 / 완속충전기대수 / 위도 / 경도 / 지원차종명

- 공공데이터포털 > 한국전력공사_전기차충전소 위경도 (~2022.07.19)
https://www.data.go.kr/data/15102458/fileData.do
<br>
- 문제 : 급속 ,완속 구분이없고, 시군구코드가 없다...

##  한국전력공사 (df)
> 한국전력공사_지역별 전기차 충전소 현황정보 (~2022.01.11)
https://home.kepco.co.kr/kepco/KE/D/htmlView/KEDDHP040.do?menuCd=FN250301

openAPI
https://bigdata.kepco.co.kr/cmsmain.do

In [115]:
import os
import sys
import urllib.request
import json

user_key = 'YjMGr00Esr0zfk7hWWsNERxF2Sa8oZ2kUVK7f3tV'

#서울만 가져옴
url = "https://bigdata.kepco.co.kr/openapi/v1/EVcharge.do?metroCd=11&apiKey="+user_key+"&returnType=json" #JSON, XML
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    response_dict = json.loads(response_body.decode('utf-8')) #json으로 stf > dict로 만들어주기
else:
    print("Error Code:" + rescode)

items = response_dict['data']
df = pd.DataFrame(items)
df.head()

,metro,city,stnPlace,stnAddr,rapidCnt,slowCnt,carType
0,서울특별시,강남구,LH강남힐스테이트,서울특별시 강남구 자곡동 자곡로3길 21,1,4,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
1,서울특별시,강남구,LH서울지사,서울특별시 강남구 선릉로 121길 12,1,0,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
2,서울특별시,강남구,가람아파트,서울특별시 강남구 일원동 일원로 127,1,0,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
3,서울특별시,강남구,강남구청 공영주차장,서울특별시 강남구 삼성동 16-1,1,0,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
4,서울특별시,강남구,강남데시앙파크,서울특별시 강남구 세곡동 헌릉로590길 63,1,1,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."


### 컬럼 정렬 및 추가

- 컬럼 배치<br>

<안심구역 데이터> <br>
광역시도코드 / 광역시도명 /시군구코드 / 시군구명 / 설치장소명 / 주소 / 급속충전기대수 / 완속충전기대수 / 위도 / 경도 / 지원차종명

In [116]:
#부족한 컬럼채우기
df['광역시도코드'] = 1 #광역시도코드
df['시군구코드'] = 1 #시군구코드
df['위도'] =1
df['경도'] =1

#이름 설정
df = df.rename(columns = {'metro':'광역시도명',
                         'city':'시군구명',
                         'stnPlace':'설치장소명',
                         'stnAddr':'주소',
                         'rapidCnt':'급속충전기대수',
                         'slowCnt':'완속충전기대수',
                         'carType':'지원차종명'})

- 컬럼추가 : 총 충전소 수( 급속 + 완속 )

In [117]:
df['충전기대수'] = df['급속충전기대수'] + df['완속충전기대수']
df.head()

,광역시도명,시군구명,설치장소명,주소,급속충전기대수,완속충전기대수,지원차종명,광역시도코드,시군구코드,위도,경도,충전기대수
0,서울특별시,강남구,LH강남힐스테이트,서울특별시 강남구 자곡동 자곡로3길 21,1,4,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E...",1,1,1,1,5
1,서울특별시,강남구,LH서울지사,서울특별시 강남구 선릉로 121길 12,1,0,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E...",1,1,1,1,1
2,서울특별시,강남구,가람아파트,서울특별시 강남구 일원동 일원로 127,1,0,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E...",1,1,1,1,1
3,서울특별시,강남구,강남구청 공영주차장,서울특별시 강남구 삼성동 16-1,1,0,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E...",1,1,1,1,1
4,서울특별시,강남구,강남데시앙파크,서울특별시 강남구 세곡동 헌릉로590길 63,1,1,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E...",1,1,1,1,2


In [118]:
#컬럼 순서배치
df = df.reindex(columns = ['광역시도코드','광역시도명','시군구코드','시군구명','설치장소명','주소','급속충전기대수','완속충전기대수','충전기대수','위도','경도','지원차종명'])
df.head()

,광역시도코드,광역시도명,시군구코드,시군구명,설치장소명,주소,급속충전기대수,완속충전기대수,충전기대수,위도,경도,지원차종명
0,1,서울특별시,1,강남구,LH강남힐스테이트,서울특별시 강남구 자곡동 자곡로3길 21,1,4,5,1,1,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
1,1,서울특별시,1,강남구,LH서울지사,서울특별시 강남구 선릉로 121길 12,1,0,1,1,1,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
2,1,서울특별시,1,강남구,가람아파트,서울특별시 강남구 일원동 일원로 127,1,0,1,1,1,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
3,1,서울특별시,1,강남구,강남구청 공영주차장,서울특별시 강남구 삼성동 16-1,1,0,1,1,1,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
4,1,서울특별시,1,강남구,강남데시앙파크,서울특별시 강남구 세곡동 헌릉로590길 63,1,1,2,1,1,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."


- 공백 데이터 전처리

In [119]:
df[df['시군구명']==' ']

,광역시도코드,광역시도명,시군구코드,시군구명,설치장소명,주소,급속충전기대수,완속충전기대수,충전기대수,위도,경도,지원차종명
618,1,서울특별시,1,,서울특별시은평병원 주차장,서울특별시 은평구 백련산로 90,1,0,1,1,1,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
619,1,서울특별시,1,,양재리본타워2단지아파트,서울특별시 서초구 양재동 212,0,4,4,1,1,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
620,1,서울특별시,1,,응암3동 공영주차장,"서울특별시 은평구 불광천길 436 (응암동, 응암공영주차장)",1,0,1,1,1,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."


In [120]:
df.loc[618,'시군구명'] = "은평구"
df.loc[619,'시군구명'] = "서초구"
df.loc[620,'시군구명'] = "은평구"

In [121]:
df[df['시군구명']==' ']

,광역시도코드,광역시도명,시군구코드,시군구명,설치장소명,주소,급속충전기대수,완속충전기대수,충전기대수,위도,경도,지원차종명


### 충전기 대수 정렬

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621 entries, 0 to 620
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   광역시도코드   621 non-null    int64 
 1   광역시도명    621 non-null    object
 2   시군구코드    621 non-null    int64 
 3   시군구명     621 non-null    object
 4   설치장소명    621 non-null    object
 5   주소       621 non-null    object
 6   급속충전기대수  621 non-null    int64 
 7   완속충전기대수  621 non-null    int64 
 8   충전기대수    621 non-null    int64 
 9   위도       621 non-null    int64 
 10  경도       621 non-null    int64 
 11  지원차종명    621 non-null    object
dtypes: int64(7), object(5)
memory usage: 58.3+ KB


In [123]:
df_group = df.groupby('시군구명')
df_charger_num = df_group[['급속충전기대수','완속충전기대수','충전기대수','위도']].sum()
df_charger_num

,급속충전기대수,완속충전기대수,충전기대수,위도
시군구명,,,,
강남구,38,81,119,54
강동구,30,56,86,34
강북구,12,16,28,9
강서구,24,43,67,27
관악구,11,17,28,16
광진구,17,18,35,12
구로구,23,52,75,30
금천구,8,2,10,6
노원구,35,62,97,42


In [124]:
#총 충전기대수 정렬
df_total = df_charger_num.sort_values(by='충전기대수', ascending=False)
df_total.reset_index(inplace=True)
df_total

,시군구명,급속충전기대수,완속충전기대수,충전기대수,위도
0,송파구,43,89,132,45
1,서초구,38,83,121,51
2,강남구,38,81,119,54
3,노원구,35,62,97,42
4,마포구,29,68,97,34
5,강동구,30,56,86,34
6,영등포구,25,60,85,25
7,용산구,27,49,76,22
8,구로구,23,52,75,30
9,도봉구,27,44,71,35


In [125]:
#급속기준 정렬
df_fast = df_charger_num.sort_values(by='급속충전기대수', ascending=False)
df_fast.reset_index(inplace=True)
df_fast

,시군구명,급속충전기대수,완속충전기대수,충전기대수,위도
0,송파구,43,89,132,45
1,강남구,38,81,119,54
2,서초구,38,83,121,51
3,성동구,36,30,66,18
4,노원구,35,62,97,42
5,중구,31,22,53,19
6,강동구,30,56,86,34
7,마포구,29,68,97,34
8,성북구,27,42,69,25
9,도봉구,27,44,71,35


In [102]:
#완속기준 정렬
df_slow = df_charger_num.sort_values(by='완속충전기대수', ascending=False)
df_slow.reset_index(inplace=True)
df_slow

,시군구명,급속충전기대수,완속충전기대수,위도
0,송파구,43,89,45
1,서초구,38,83,51
2,강남구,38,81,54
3,마포구,29,68,34
4,노원구,35,62,42
5,영등포구,25,60,25
6,강동구,30,56,34
7,구로구,23,52,30
8,용산구,27,49,22
9,도봉구,27,44,35


# 환경부 산하기관(한국수자원공사) (df2)
전국 전기차 충전소 데이터 (~2021.12.21)

https://www.bigdata-environment.kr/user/data_market/detail.do?id=2081f250-b235-11eb-bc66-992d9417bbde#!

## 컬럼 정렬 및 추가

In [109]:
%pwd

'C:\\cakd7\\github\\데이터안심구역_팀\\teamproject'

In [279]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df2 = pd.read_csv('전체_전기차충전소.csv')

In [280]:
df2.columns

Index(['stat_nm', 'stat_id', 'chger_id', 'chger_type', 'addr', 'lat', 'lon',
       'use_time', 'busi_id', 'busi_nm', 'busi_call', 'chger_stat',
       'stat_upd_dt', 'power_type', 'city_do_cd', 'parking_free_yn',
       'stat_note', 'limit_yn', 'limit_detail', 'del_yn', 'del_detail'],
      dtype='object')

In [281]:
#한글 컬럼명으로 변환
df2 = df2.rename(columns = {'stat_nm':'충전소명',
                    'stat_id':'충전소ID',
                    'chger_id':'충전기ID',
                    'chger_type':'충전기타입',
                    'addr':'주소',
                    'lat':'위도',
                    'lon':'경도',
                    'use_time':'이용가능시간',
                    'busi_id':'기관아이디',
                    'busi_nm':'운영기관명',
                    'busi_call':'운영기관 연락처',
                    'chger_stat':'충전기상태',
                    'stat_upd_dt':'상태갱신일시',
                    'power_type':'충전량유형',
                    'city_do_cd':'시도코드',
                    'parking_free_yn':'주차료무료여부',
                    'stat_note':'충전소안내',
                    'limit_yn':'이용자제한여부',
                    'limit_detail':'이용제한사유',
                    'del_yn':'삭제여부',
                    'del_detail':'삭제사유'})
df2.head()

,충전소명,충전소ID,충전기ID,충전기타입,주소,위도,경도,이용가능시간,기관아이디,운영기관명,...,충전기상태,상태갱신일시,충전량유형,시도코드,주차료무료여부,충전소안내,이용자제한여부,이용제한사유,삭제여부,삭제사유
0,종묘 공영주차장,ME000001,1,3,서울특별시 종로구 종로 157,37.571076,126.995880,24시간 이용가능,ME,환경부,...,9,2.021111e+13,NaN,11,Y,NaN,N,NaN,N,NaN
1,세종로 공영주차장,ME000002,1,6,서울특별시 종로구 세종대로 189,37.573611,126.976011,24시간 이용가능,ME,환경부,...,9,2.021083e+13,NaN,11,Y,NaN,N,NaN,N,NaN
2,그랜드앰배서더 서울,ME000003,1,6,서울특별시 중구 동호로 287,37.559352,127.002350,24시간 이용가능,ME,환경부,...,9,2.021091e+13,NaN,11,N,NaN,N,NaN,N,NaN
3,한강진역 공영주차장,ME000004,1,3,서울특별시 용산구 한남동 산10-84,37.540085,127.002804,24시간 이용가능,ME,환경부,...,9,2.021091e+13,NaN,11,Y,NaN,N,NaN,N,NaN
4,기아차 성동서비스센터,ME000005,1,3,서울특별시 성동구 성수이로26길 61,37.544834,127.064226,24시간 이용가능,ME,환경부,...,9,2.021072e+13,NaN,11,N,20190626 공사로 인한 임시철거,Y,20190626 공사로 인한 임시철거,N,NaN


- 필요컬럼 뽑기 <br>
<한전데이터> <br>
['광역시도코드','광역시도명','시군구코드','시군구명','설치장소명','주소','급속충전기대수','완속충전기대수','위도','경도','지원차종명','충전기대수']

In [282]:
#서울만 뽑기 - 시도코드=11 #총13388
df2 = df2[df2['시도코드']==11]
#인덱스정렬
df2.reset_index(inplace=True)
df2

,index,충전소명,충전소ID,충전기ID,충전기타입,주소,위도,경도,이용가능시간,기관아이디,...,충전기상태,상태갱신일시,충전량유형,시도코드,주차료무료여부,충전소안내,이용자제한여부,이용제한사유,삭제여부,삭제사유
0,0,종묘 공영주차장,ME000001,1,3,서울특별시 종로구 종로 157,37.571076,126.995880,24시간 이용가능,ME,...,9,2.021111e+13,NaN,11,Y,NaN,N,NaN,N,NaN
1,1,세종로 공영주차장,ME000002,1,6,서울특별시 종로구 세종대로 189,37.573611,126.976011,24시간 이용가능,ME,...,9,2.021083e+13,NaN,11,Y,NaN,N,NaN,N,NaN
2,2,그랜드앰배서더 서울,ME000003,1,6,서울특별시 중구 동호로 287,37.559352,127.002350,24시간 이용가능,ME,...,9,2.021091e+13,NaN,11,N,NaN,N,NaN,N,NaN
3,3,한강진역 공영주차장,ME000004,1,3,서울특별시 용산구 한남동 산10-84,37.540085,127.002804,24시간 이용가능,ME,...,9,2.021091e+13,NaN,11,Y,NaN,N,NaN,N,NaN
4,4,기아차 성동서비스센터,ME000005,1,3,서울특별시 성동구 성수이로26길 61,37.544834,127.064226,24시간 이용가능,ME,...,9,2.021072e+13,NaN,11,N,20190626 공사로 인한 임시철거,Y,20190626 공사로 인한 임시철거,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13383,88310,삼성전자 서초사옥,TDA00018,4,2,서울특별시 서초구 서초대로74길 11,37.496633,127.026915,05:00~24:00,TD,...,2,2.021113e+13,NaN,11,Y,NaN,Y,거주자외 사용제한,N,NaN
13384,88311,삼성전자 서초사옥,TDA00018,5,2,서울특별시 서초구 서초대로74길 11,37.496633,127.026915,05:00~24:00,TD,...,2,2.021113e+13,NaN,11,Y,NaN,Y,거주자외 사용제한,N,NaN
13385,88312,삼성전자 서초사옥,TDA00018,6,2,서울특별시 서초구 서초대로74길 11,37.496633,127.026915,05:00~24:00,TD,...,9,NaN,NaN,11,Y,NaN,Y,거주자외 사용제한,N,NaN
13386,88313,삼성전자 서초사옥,TDA00018,7,2,서울특별시 서초구 서초대로74길 11,37.496633,127.026915,05:00~24:00,TD,...,2,2.021113e+13,NaN,11,Y,NaN,Y,거주자외 사용제한,N,NaN


In [283]:
#컬럼선택
df2 =df2[['충전소명','충전기타입','주소','위도','경도','운영기관명','시도코드']]
#컬럼명 변경
df2.rename(columns = {'시도코드':'광역시도코드',
                     '충전소명':'설치장소명',},inplace=True)

#컬럼추가
df2['광역시도명'] = '서울특별시'
df2['시군구코드'] = 1
df2['시군구명'] = 1
df2['급속충전기대수'] = 0
df2['완속충전기대수'] = 0
df2['급속완속충전기대수'] = 0
df2['충전기대수'] = 0
df2['지원차종명'] = 1

## 주소 결측치, 구 설정

In [284]:
#null값 확인 
# df2.info()

#주소에 null값 존재
df2[df2['주소'].isnull()==True]

df2.loc[99:101,'주소'] = '서울 금천구 두산로 71'
df2.loc[151,'주소'] = '서울 은평구 은평로 111'
#가락시장은 가락농수산물종합도매시장으로 검색했음 
df2.loc[269:275,'주소'] = '서울 송파구 양재대로 932'


df2[df2['주소'].isnull()==True]

,설치장소명,충전기타입,주소,위도,경도,운영기관명,광역시도코드,광역시도명,시군구코드,시군구명,급속충전기대수,완속충전기대수,급속완속충전기대수,충전기대수,지원차종명


In [285]:
import re
add = df2['주소'].to_list()
for i in range(len(add)):
    m =re.search('\w+구',str(add[i]))    
    if m is not None:
        print(m.group())
    else:
        print('No matches')
    df2.loc[i,'시군구명'] = m.group()

# type(val)

종로구
종로구
중구
용산구
성동구
성동구
동대문구
동대문구
중랑구
성북구
성북구
강북구
도봉구
은평구
은평구
마포구
마포구
마포구
양천구
양천구
강서구
강서구
강서구
구로구
구로구
구로구
금천구
금천구
금천구
영등포구
동작구
동작구
관악구
서초구
서초구
강남구
강남구
강남구
강남구
강남구
송파구
강동구
강남구
강남구
강동구
강동구
강동구
강서구
강서구
강서구
강서구
강서구
관악구
관악구
관악구
관악구
관악구
입구
관악구
광진구
구로구
노원구
노원구
노원구
노원구
동대문구
동작구
마포구
No matches


AttributeError: 'NoneType' object has no attribute 'group'

In [286]:
df2.head()

,설치장소명,충전기타입,주소,위도,경도,운영기관명,광역시도코드,광역시도명,시군구코드,시군구명,급속충전기대수,완속충전기대수,급속완속충전기대수,충전기대수,지원차종명
0,종묘 공영주차장,3,서울특별시 종로구 종로 157,37.571076,126.995880,환경부,11,서울특별시,1,종로구,0,0,0,0,1
1,세종로 공영주차장,6,서울특별시 종로구 세종대로 189,37.573611,126.976011,환경부,11,서울특별시,1,종로구,0,0,0,0,1
2,그랜드앰배서더 서울,6,서울특별시 중구 동호로 287,37.559352,127.002350,환경부,11,서울특별시,1,중구,0,0,0,0,1
3,한강진역 공영주차장,3,서울특별시 용산구 한남동 산10-84,37.540085,127.002804,환경부,11,서울특별시,1,용산구,0,0,0,0,1
4,기아차 성동서비스센터,3,서울특별시 성동구 성수이로26길 61,37.544834,127.064226,환경부,11,서울특별시,1,성동구,0,0,0,0,1


## 급속, 완속 분류

> 충전기 타입 <br>
1) 급속 <br>
01:DC차데모 / 03: DC차데모+AC3상(급속) / 04: DC콤보,05: DC차데모+DC콤보 /06: DC차데모+AC3상+DC콤보 <br>
2) 완속 <br>
02: AC완속 <br>
3) 완속, 급속 <br>
07: AC3상(급속+완속)	<br>

> 전기차 충전기 종류 <br> 
https://tago.kr/story/charger.htm <br>
http://wiki.hash.kr/index.php/%EC%B0%A8%EB%8D%B0%EB%AA%A8 <BR>
> 급속충전기(DC-직류), 완속충전기(AC)

In [271]:
df2['충전기타입'].unique()

array([3, 6, 4, 5, 2, 7, 1], dtype=int64)

In [287]:
for ind, val in enumerate(df2['충전기타입']):
    if val in (1,3,4,5,6):
        df2.loc[ind,'급속충전기대수'] = 1
    elif val == 2:
        df2.loc[ind,'완속충전기대수']= 1
    elif val==7:
         df2.loc[ind,'급속완속충전기대수']= 1

## df2

- 주소별로 합산

In [296]:
df3 = df2.copy()

In [297]:
df3 = df3.groupby('설치장소명')

In [299]:
df3['급속충전기대수'] = df3['급속충전기대수'].sum()
# ,'완속충전기대수','급속완속충전기대수']
df3

TypeError: 'DataFrameGroupBy' object does not support item assignment

In [293]:
df2.head()

,설치장소명,충전기타입,주소,위도,경도,운영기관명,광역시도코드,광역시도명,시군구코드,시군구명,급속충전기대수,완속충전기대수,급속완속충전기대수,충전기대수,지원차종명
0,종묘 공영주차장,3,서울특별시 종로구 종로 157,37.571076,126.995880,환경부,11,서울특별시,1,종로구,1,0,0,0,1
1,세종로 공영주차장,6,서울특별시 종로구 세종대로 189,37.573611,126.976011,환경부,11,서울특별시,1,종로구,1,0,0,0,1
2,그랜드앰배서더 서울,6,서울특별시 중구 동호로 287,37.559352,127.002350,환경부,11,서울특별시,1,중구,1,0,0,0,1
3,한강진역 공영주차장,3,서울특별시 용산구 한남동 산10-84,37.540085,127.002804,환경부,11,서울특별시,1,용산구,1,0,0,0,1
4,기아차 성동서비스센터,3,서울특별시 성동구 성수이로26길 61,37.544834,127.064226,환경부,11,서울특별시,1,성동구,1,0,0,0,1


In [256]:
#컬럼순서
df2=df2.reindex(columns =['광역시도코드','광역시도명','시군구코드','시군구명','설치장소명','주소','급속충전기대수','완속충전기대수','충전기대수','위도','경도','지원차종명'])
df2.head()

,광역시도코드,광역시도명,시군구코드,시군구명,설치장소명,주소,급속충전기대수,완속충전기대수,충전기대수,위도,경도,지원차종명
0,11,서울특별시,1,1,종묘 공영주차장,서울특별시 종로구 종로 157,1,1,1,37.571076,126.995880,1
1,11,서울특별시,1,1,세종로 공영주차장,서울특별시 종로구 세종대로 189,1,1,1,37.573611,126.976011,1
2,11,서울특별시,1,1,그랜드앰배서더 서울,서울특별시 중구 동호로 287,1,1,1,37.559352,127.002350,1
3,11,서울특별시,1,1,한강진역 공영주차장,서울특별시 용산구 한남동 산10-84,1,1,1,37.540085,127.002804,1
4,11,서울특별시,1,1,기아차 성동서비스센터,서울특별시 성동구 성수이로26길 61,1,1,1,37.544834,127.064226,1


# 행정안전부
전기차 충전소 데이터
https://www.data.go.kr/data/15101898/openapi.do <br>

1PUGYUKR-1PUG-1PUG-1PUG-1PUGYUKRA0

In [56]:
import os
import sys
import urllib.request
import json
import pandas as pd

user_key = '1PUGYUKR-1PUG-1PUG-1PUG-1PUGYUKRA0'

url= "http://www.safemap.go.kr/legend/legendApiXml.do?apikey="+ user_key + "&layer=A2SM_CHARGINGSTATION&style=A2SM_CHARGINGSTATION"

request = urllib.request.Request(url)
response = urllib.request.urlopen(request)
response_body = response.read()
# response_dict = json.loads(response_body.decode('utf-8')) #json으로 stf > dict로 만들어주기

# items = response_dict['data']
# df = pd.DataFrame(items)
# df.head()

In [57]:
response_body
response_body.decode('utf-8')

'<?xml version="1.0" encoding="utf-8"?>\r\n\r\n\r\n\r\n\r\n\r\n\r\n<data>\r\n\t<legend>\r\n\t\r\n\t\t<LAYER>A2SM_CHARGINGSTATION</LAYER>\r\n\t\t<STYLE>A2SM_CHARGINGSTATION</STYLE>\r\n\t\t\r\n\t\t\r\n\t\t<CATEGORY>안전시설</CATEGORY>\r\n\t\t\r\n\t\t<NAME>전기충전소</NAME>\r\n\t\t<STDRDE>2021년 3월</STDRDE>\r\n\t\t<PROVIDE>한국환경공단</PROVIDE>\r\n\t\t<WARN></WARN>\r\n\t\t<HELP></HELP>\r\n\t\t<DETAIL>\r\n\t\t\r\n\t\t\t<item>\r\n\t\t\t\t\r\n\t\t\t\t<IMG_NM>poi20.png</IMG_NM>\r\n\t\t\t\t<LGD_NO>1</LGD_NO>\r\n\t\t\t\t<LGD_NM>전기충전소</LGD_NM>\r\n\t\t\t\t<LGD_RANGE></LGD_RANGE>\r\n\t\t\t</item>\r\n\t\t\r\n\t\t</DETAIL>\r\n\t\r\n\t\r\n\t</legend>\r\n</data>\r\n'